### 导包

In [1]:
import requests # 发起网络请求
import json  # 处理json格式的数据
import time  # 时间
import random  # 随机
import re  # 正则表达式

### 淘宝天猫
网址：https://detail.tmall.com/item.htm?spm=a230r.1.14.14.385d7868FK4F41&id=589815124915&cm_id=140105335569ed55e27b&abbucket=14&on_comment=1&sku_properties=10004:827902415;5919063:6536025

In [2]:
num=1
while True:
    url='https://rate.tmall.com/list_detail_rate.htm?'
    headers={
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36',
        'cookie': 'cna=M6ZIFXAQVQICATr9Rfw6Hdh2; lid=wei5741679; hng=CN%7Czh-CN%7CCNY%7C156; t=6c016af0e470a8eabfca45da32ee0de1; tracknick=wei5741679; lgc=wei5741679; enc=w9fVxl%2BWgbpQZ1zt%2ByBOAy%2BMQB1gSyr2TC56LcAWYXAeNk87Y5dNoHuX5ZESYkkqliBfpnnjPK%2BSiXWnnR%2BYCw%3D%3D; _tb_token_=5e1b0a3713733; cookie2=12b7eb27a0cfe647cb767db955734031; x5sec=7b22726174656d616e616765723b32223a223761366332623136363637303762626131333435613163613431343463366234434e435776664d46454c506b3171584b3075656e48773d3d227d; l=dBxET1tuqlDyXuEBBOfwdFyish_O2nRbnsPyQ6oeHIB1tZOUJdzD1HwElIdAQ3QQEtfbGetzO3MF-Rn25q4LRvMDBeYCKXIpBxv9R; isg=BAsLGDXi0b3E3Q49oLjaxB_xmq_1oB8iqRweSn0YWM5cnHX-BXLycnx-dpxyvHca; dnk=wei5741679; uc1=cookie16=U%2BGCWk%2F74Mx5tgzv3dWpnhjPaQ%3D%3D&cookie21=U%2BGCWk%2F7pY%2FF&cookie15=U%2BGCWk%2F75gdr5Q%3D%3D&existShop=false&pas=0&cookie14=UoTUPvOOt3Wz6w%3D%3D&tag=8&lng=zh_CN; uc3=id2=UUGk2tWq9%2B8n6Q%3D%3D&nk2=FPzaAcj%2BZGcEZA%3D%3D&lg2=V32FPkk%2Fw0dUvg%3D%3D&vt3=F8dBxd7OsV6CfHOanOY%3D; uc4=id4=0%40U2OT6jwMoU7GPXsRrBy8%2FBib5VT8&nk4=0%40Fne9c8hIf4FUGWZDvU7Neb%2BKbjLk; _l_g_=Ug%3D%3D; unb=2964045587; cookie1=Vyu9KgDVKvtz%2FT8BpsCMgguQz22CyYUmFVQy45qu3e0%3D; login=true; cookie17=UUGk2tWq9%2B8n6Q%3D%3D; _nk_=wei5741679; sgcookie=E3pwOp4RICXEUblDKuz1g; sg=975; csg=92223529',
        'referer': 'https://detail.tmall.com/item.htm?spm=a230r.1.14.6.591c7868vyGMde&id=589815124915&cm_id=140105335569ed55e27b&abbucket=14&sku_properties=10004:827902415;5919063:6536025'
    }
    params={
        'itemId': '589815124915',
        'spuId': '1182940081',
        'sellerId': '2838892713',
        'order': '3', # 1表示按时间排序，3表示默认排序
        'currentPage': num,
        'append': '1',  # 0：默认，1：只显示有追评的
        'content': '1', # 1：只显示有内容，0：包含无内容
        'needFold': '1'
    }
    r=requests.get(url,params=params,headers=headers)
    temp=json.loads(re.sub('^json.*?\(','',r.text.strip()).replace(')',''))
    lastnum=temp['rateDetail']['paginator']['lastPage']
    time.sleep(random.uniform(1.5,2))
    with open(f'./data/tm/默认追评fold{num}.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(temp,indent=4,ensure_ascii=False))
    print(f'第{num}次爬取成功！')
    num+=1
    if num > lastnum:
        break

### 京东商城
网址：https://item.jd.com/100002795957.html#none

In [3]:
scores={'全部':0,'差评':1,'中评':2,'好评':3,'追评':5,'视频':7}
sorts={'推荐':5,'时间':6}
for key1,sort in sorts.items():
    for key2,score in scores.items():
        page=0
        while True:
            url='https://club.jd.com/comment/productPageComments.action?'
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
            }
            params={
                'productId': '100002795957',
                'score': score,  # 0：全部，1：差评，2：中评，3：好评，4：晒图，5：追评，7：视频晒单
                'sortType': sort, # 5：推荐排序，6：时间排序
                'page': page,
                'pageSize': '10',
                'isShadowSku': '0',
                'fold': '1'
            }
            r=requests.get(url,params=params,headers=headers)
            temp=r.json()
            time.sleep(random.uniform(1.5,2))
            if temp['comments'] == []:
                break
            with open(f'./data/jd/{key1}{key2}{page}.json', 'w', encoding='utf-8') as f:
                f.write(json.dumps(temp,indent=4,ensure_ascii=False))
            print(f'{key1} {key2} 第{page}次爬取成功！')
            page+=1

### 苏宁易购
网址：https://product.suning.com/0000000000/10973073658.html?safp=d488778a.13701.productWrap.2&safc=prd.3.ssdsn_pic00-1_jz&safpn=10007

In [4]:
cates={'全部':'total','最新':'newest','有图':'show','好评':'good','中评':'normal','差评':'bad','追评':'again'}
mobiles={'黑8+128':'73658','黑8+256':'73783','蓝8+128':'73686','蓝8+256':'73813'}
for mobile,code in mobiles.items():
    for name,cate in cates.items():
        num=1
        while True:
            url='https://review.suning.com/ajax/cluster_review_lists/general-30355545-0000000109730{0}-0000000000-{1}-{2}-default-10-----reviewList.htm?callback=reviewList'.format(code,cate,num)
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
            }
            r=requests.get(url,headers=headers)
            temp=json.loads(r.text.replace('reviewList(','').replace(')',''))
            time.sleep(random.uniform(1.5,2))
            if 'commodityReviews' not in temp.keys():
                break
            elif temp['commodityReviews'] == []:
                break
            with open(f'./data/sn/{mobile}{name}{num}.json', 'w', encoding='utf-8') as f:
                f.write(json.dumps(temp,indent=4,ensure_ascii=False))
            print(f'{mobile}{name}  第{num}页爬取成功！')
            num+=1

### 华为商城
网址：https://www.vmall.com/product/10086102942203.html?cid=70146

In [5]:
page=1
while True:
    url='https://openapi.vmall.com/rms/comment/getCommentList.json?t=1584525816839'
    headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
    }
    data={
        "pid":"10086102942203",
        "gbomCode":"",
        "extraType":"0",  # 0:全部，1：热门，2：有图
        "pageSize":'20',
        "pageNum":page
    }
    r=requests.post(url,json=data,headers=headers)
    temp=r.json()
    time.sleep(random.uniform(1.5,2))
    if temp['data']['comments']==[]:
        break
    with open(f'./data/hw/{page}.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(temp,indent=4,ensure_ascii=False))
    print(f'第{page}次爬取成功！')
    page+=1